# TOPIC NAME

## Introduction

This guide will show you how to:

* TODO
* TODO

By the end of it, you will TODO!

## Setup

Install dependencies

In [ ]:
pip install neptune-client TODO

## Step 1: TODO

TODO

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

torch.manual_seed(0)

# create data
import numpy as np
from sklearn.datasets import make_classification

X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)


# create pytorch module
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.dropout = dropout

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X


# create neptune logger and pass it to NeuralNetClassifier
from skorch import NeuralNetClassifier
import neptune
from skorch.callbacks.logging import NeptuneLogger

neptune.init(api_token='ANONYMOUS',
             project_qualified_name='shared/skorch-integration')
experiment = neptune.create_experiment(
    name='skorch-basic-example',
    params={'max_epochs': 20,
            'lr': 0.1},
    upload_source_files=['skorch_example.py'])
neptune_logger = NeptuneLogger(experiment, close_after_train=False)

net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    callbacks=[neptune_logger]
)

# run training
net.fit(X, y)

# log score after training
from sklearn.metrics import roc_auc_score

y_pred = net.predict_proba(X)
auc = roc_auc_score(y, y_pred[:, 1])

neptune_logger.experiment.log_metric('roc_auc_score', auc)

# log charts like ROC curve
from scikitplot.metrics import plot_roc
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16, 12))
plot_roc(y, y_pred, ax=ax)
neptune_logger.experiment.log_image('roc_curve', fig)

# log model after training
net.save_params(f_params='basic_model.pkl')
neptune_logger.experiment.log_artifact('basic_model.pkl')

# close experiment
neptune_logger.experiment.stop()

## Step 2: Initialize Neptune

Connects your script to Neptune application. 

In [ ]:
import neptune

neptune.init(
    api_token="ANONYMOUS",
    project_qualified_name="shared/onboarding"
)

You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune project `project_qualified_name`.

---

**Note:** 


Instead of logging data to the public project 'shared/onboarding' as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your Neptune API token

![image](https://neptune.ai/wp-content/uploads/get_token.gif)

2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your username to the ``project_qualified_name`` argument of the ``neptune.init()`` method: ``project_qualified_name='YOUR_USERNAME/sandbox``. Keep `/sandbox` at the end, the `sandbox` project that was automatically created for you.

For example:

```python
neptune.init(project_qualified_name='funky_steve/sandbox', 
             api_token='eyJhcGlfYW908fsdf23f940jiri0bn3085gh03riv03irn',
            )
```

---

## Step 3: Create an experiment

In [ ]:
neptune.create_experiment(name='great-idea')

This opens a new "experiment" namespace in Neptune to which you can log various objects.

Click on the link above to open this experiment in Neptune.

For now it is empty but keep the tab with experiment open to see what happens next. 

## Step 4: TODO

TODO

In [ ]:
TODO

---

**Note:**
   
When you track experiments with Neptune in Jupyter notebooks you need to explicitly stop the experiment by running `neptune.stop()`.

If you are running Neptune in regular `.py` scripts it will stop automatically when your code stops running.

---

In [ ]:
neptune.stop()